
# 

# A Quick Guide To CuPy
---

### Target Audience
The CuPy quick guide targets python developers who are interested in developing HPC applications using CUDA accelerated Python library on the GPU. A background in C programming maybe recommended for intermediate level to foster easy understanding.

### Objectives
- **The objectives of this guide are to:**
     - quickly get you started with CuPy from beginner to intermediate level
     - teach you application of GPU programming concept to HPC field(s)
     - show you how to maximize the throughput of your HPC implementation through computational speedup on the GPU.  

### Outline
   1. What is CuPy?
   2. Features of CuPy
   3. Installation Guide
   4. CuPy Fundamentals
   5. CUDA Kernels
   6. Summary
   7. HPC Approach
    


## 1.  What is CuPy?
- CuPy is an implementation of NumPy-compatible multi-dimensional array on CUDA
- CuPy consists of :
    - cupy.ndarray 
    - the core multi-dimensional array class 
    - many functions 
- It supports a subset of numpy.ndarray interface which include:
    - Basic and advance indexing 
    - Data types (int32, float32, uint64, complex64,... )
    - Array manipulation routine (reshape)
    - Linear Algebra functions (dot, matmul, etc)
    - Reduction along axis (max, sum, argmax, etc)

In [19]:
import numpy as np
X = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#Basic indexing and slicing
print(X[5:])

#output: [5 6 7 8 9]

print(X[1:7:2])

#output: [1 3 5]

#reduction and Linear Algebra function
print("max:", max(X))

#output: max: 9

[5 6 7 8 9]
[1 3 5]
max: 9


In [20]:
import numpy as np
#Advance indexing
X = np.array([[1, 2],[3, 4],[5, 6]])
print(X[[0, 1, 2], [0, 1, 0]])

#output: [1 4 5]

B = np.array([1,2,3,4], dtype=np.float32)
C = np.array([5,6,7,8], dtype=np.float32)
print("matmul:",np.matmul(B, C))

#output: matmul: 70.0

[1 4 5]
matmul: 70.0


In [21]:
import numpy as np
#data type and array manipulation routine 
A =1j*np.arange(9, dtype=np.complex64).reshape(3,3)
print(A)


#output:
#[[0.+0.j 0.+1.j 0.+2.j]
# [0.+3.j 0.+4.j 0.+5.j]
# [0.+6.j 0.+7.j 0.+8.j]]

[[0.+0.j 0.+1.j 0.+2.j]
 [0.+3.j 0.+4.j 0.+5.j]
 [0.+6.j 0.+7.j 0.+8.j]]


## 2. Features of CuPy

- **Features of CuPy includes:**
    - User-define elementwise CUDA kernels
    - User-define reduction CUDA kernels
    - Fusing CUDA kernels to optimize user-define calculation
    - Customizable memory allocator and memory pool
    - cuDNN utilities
- These features  are developed to support performance
- CuPy uses on-the-fly kernel synthesis: when a kernel call is required, it compiles a kernel code optimized for the shapes and dtypes of given arguments, sends it to the GPU device, and executes the kernel.
- CuPy also caches the kernel code sent to GPU device within the process, which reduces the kernel transfer time on further calls. 

## 3. CuPy Installation Guide

- **Requirements:**
    - Recommended Linux distributions are Centos and Ubuntu
    - NVIDIA CUDA GPU with the Compute Capability 3.0 or larger 
    - CUDA Toolkit: v9.0 - v11.2
    - Python: v3.5.1+ - v3.9.0+ 
    - CuPy can also be install Windows OS but only supports Python 3.6.0 or later
- **Python Dependencies**
    - NumPy/SciPy-compatible API in CuPy v8 is based on NumPy 1.19 and SciPy 1.5.

- **Wheels (precompiled binary package)**
    - Available for Linux (x86_64, Python 3.5+) and Windows (amd64, Python 3.6+).

- **Conda-Forge**
    - conda install -c conda-forge cupy
    - If you need to enforce the installation of a particular CUDA version (say 10.0) for driver compatibility, you can do:
        - conda install -c conda-forge cupy cudatoolkit=10.0 
    - To install CuPy with the cuTENSOR support enabled, you can do: 
        - conda install -c conda-forge cupy cutensor cudatoolkit=10.2

- **CuPy inside Docker**
    - You can pull CuPy Docker images from [here](https://hub.docker.com/r/cupy/cupy/). 
    - Using docker pull cupy/cupy
    - Use NVIDIA Container Toolkit to run CuPy image with GPU. You can login to the environment with bash, and run the Python interpreter: 
        - docker run --gpus all -it cupy/cupy /bin/bash 
        - docker run --gpus all -it cupy/cupy /usr/bin/python 

- **Conda (full RAPIDS package)**
<img src="../images/rapids_package.png">

###           CuPy Architecture

<img src="../images/cupy_arch.png">


## 4. CuPy Fundamentals
- **Cupy.ndarray**: CuPy is a GPU array backend that implements a subset of NumPy interface.
```python
#CuPy version
import cupy as cp
X_gpu = cp.array([1, 2, 3, 4, 5])
#NumPy version
import numpy as np
x = np.array([1, 2, 3, 4, 5])
```

- CuPy considers the current device as the default device with device ID 0. It also allows temporary switch between GPU devices.

```python
import cupy as cp
####### Current device (GPU ID: 0)##############
gpu_0 = cp.array([1, 2, 3, 4, 5])

# Switch device
cp.cuda.Device(1).use()
gpu_1 = cp.array([1, 2, 3, 4])

###### Switch GPU temporarily################
import numpy as np
with cp.cuda.Device(1):
      gpu_1 = cp.array([1, 2, 3, 4])
# back to device id 0
gpu0 = cp.array([1, 2, 3, 4, 5]) 
```

### Data transfer
- Arrays can be moved from Host to Device (CPU -> GPU) using **cupy.asarray**

In [22]:
import cupy as cp
import numpy as np
x = np.array([1, 2, 3, 4, 5])
x_gpu = cp.asarray(x)
print(x_gpu)

#output: [1 2 3 4 5]

[1 2 3 4 5]


- Device array can be move to Host(GPU -> CPU) using: **cupy.asnumpy or cupy.ndarray.get()**


In [23]:
import cupy as cp
import numpy as np
x_gpu = cp.array([1, 2, 3, 4, 5])
#copy to Host
x_cpu = cp.asnumpy(x_gpu)
print("x_cpu: ",x_cpu)

#alternative option
x_cpu_alt = x_gpu.get()
print("x_cpu_alt: ",x_cpu_alt) 

#output: 
#x_cpu:  [1 2 3 4 5]
#x_cpu_alt:  [1 2 3 4 5]

x_cpu:  [1 2 3 4 5]
x_cpu_alt:  [1 2 3 4 5]


- In order to transfer an array between devices(GPU to GPU), **cupy.ndarray** is used.

In [24]:
import cupy as cp
with cp.cuda.Device(0):
    x_gpu_0 = cp.ndarray([ 2, 3, 3]) 
print("x_gpu_0:\n", x_gpu_0)

with cp.cuda.Device(0):
      x_gpu_1 = cp.asarray(x_gpu_0)
print("x_gpu_1:\n", x_gpu_1)

#output
#x_gpu_0:
# [[[0. 0. 0.]
#  [0. 0. 0.]
#  [0. 0. 0.]]

# [[0. 0. 0.]
#  [0. 0. 0.]
#  [0. 0. 0.]]]


x_gpu_0:
 [[[0.00000000e+00 2.00000047e+00 5.12000122e+02]
  [8.19200196e+03 1.31072031e+05 5.24288126e+05]
  [2.09715251e+06 8.38861003e+06 3.35544401e+07]]

 [[6.71088803e+07 1.34217761e+08 2.68435521e+08]
  [5.36871042e+08 1.07374209e+09 2.14748417e+09]
  [4.29496834e+09 8.58993669e+09 1.28849040e+10]]]
x_gpu_1:
 [[[0.00000000e+00 2.00000047e+00 5.12000122e+02]
  [8.19200196e+03 1.31072031e+05 5.24288126e+05]
  [2.09715251e+06 8.38861003e+06 3.35544401e+07]]

 [[6.71088803e+07 1.34217761e+08 2.68435521e+08]
  [5.36871042e+08 1.07374209e+09 2.14748417e+09]
  [4.29496834e+09 8.58993669e+09 1.28849040e+10]]]


### GPU  & CPU agnostic code
- The compatibility of CuPy with NumPy enables the implementation of CPU/GPU generic code using **cupy.get_array_module()**

In [25]:
import cupy as cp
import numpy as np

#example: log(1 + exp(x))
x_cpu  = np.array([1, 2, 3, 4, 5])
x_gpu  = cp.get_array_module(x_cpu)
result = x_gpu.maximum(0, x_cpu) + x_gpu.log1p(x_gpu.exp(-abs(x_cpu)))
print(result)

#output: [1.31326169 2.12692801 3.04858735 4.01814993 5.00671535]

#An explicit conversion to a host 
x_gpu  = cp.array([6, 7, 8, 9, 10])
result = cp.asnumpy(x_gpu) + x_cpu
print(result)

#output: [ 7  9 11 13 15]

#An explicit conversion to a device
result = x_gpu + cp.asarray(x_cpu)
print(result)

#output: [ 7  9 11 13 15]


[1.31326169 2.12692801 3.04858735 4.01814993 5.00671535]
[ 7  9 11 13 15]
[ 7  9 11 13 15]


## 5. CUDA Kernels

- **CUDA Kernels can be define in Cupy as follows:**

    - Elementwise Kernels
    - Reduction Kernels
    - Raw Kernels
    - Kernel Fusion
- These kernels are user-defined based.

### Elementwise Kernels
- The ElementwiseKernel class is used to define this type of kernel.
- This kernel consists of four parts which includes:
    1. a list of input argument 
    2. a list of output argument
    3. a loop body code
    4. kernel name
- Variable name starting with underscore “_” , “n”, and “i” are regarded as reserved keywords.

#### Example: z = x*w + b

In [26]:
import cupy as cp

input_list = 'float32 x , float32 w, float32 b'
output_list = 'float32 z'
code_body  = 'z =  (x * w) + b'

# elementwisekernel class defined
dnnLayerNode = cp.ElementwiseKernel(input_list, output_list, code_body,'dnnLayerNode')

# data
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
b = cp.array([-0.5], dtype=cp.float32)
z = cp.empty((3,3), dtype=cp.float32)

# kernel call with argument passing
dnnLayerNode(x,w,b,z)
print(z)

#output:
#[[-0.5  0.5  3.5]
# [ 8.5 15.5 24.5]
# [35.5 48.5 63.5]]


[[-0.5  0.5  3.5]
 [ 8.5 15.5 24.5]
 [35.5 48.5 63.5]]


### Elementwise Kernel: Generic-type kernels
- It can be used to define a generic-type kernels. It treats a type specifier of one character as a type placeholder.

In [27]:
import cupy as cp

input_list = 'T x , T w, T b'
output_list = 'T z'
code_body  = 'z =  (x * w) + b'

# elementwisekernel class defined
dnnLayerNode = cp.ElementwiseKernel(input_list, output_list, code_body,'dnnLayerNode')
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
b = cp.array([-0.5], dtype=cp.float32)
z = cp.empty((3,3), dtype=cp.float32)

# kernel call with argument passing
dnnLayerNode(x,w,b,z)
print(z)

#output:
#[[-0.5  0.5  3.5]
# [ 8.5 15.5 24.5]
# [35.5 48.5 63.5]]

[[-0.5  0.5  3.5]
 [ 8.5 15.5 24.5]
 [35.5 48.5 63.5]]


### Reduction Kernels

- Reduction kernel is implemented through the ReductionKernel class. 
- In order to implement this kernel class, the following parts must be defined:
    - **Identity value**: to initialize reduction value.
    - **Mapping expression**: Used for the pre-processing of each element to be reduced.
    - **Reduction expression**: It is an operator to reduce the multiple mapped values. The special variables **a** and **b** are used for its operands.
    - **Post mapping expression**: It is used to transform the resulting reduced values. The special variable a is used as its input. Output should be written to the output parameter.


**Example:  𝑧=∑_(𝑖=1)𝑥_𝑖 *𝑤_𝑖+𝑏**

In [28]:
import cupy as cp
dnnLayer = cp.ReductionKernel(
	'T x, T w, T bias',              
	'T z',                         
	'x * w',                      
	'a + b', 
	'z = a + bias',              
	'0',                            
	'dnnLayer'  )
x = cp.arange(10, dtype=np.float32).reshape(2,5)
w = cp.arange(10, dtype=np.float32).reshape(2,5)
bias = -0.1
z = dnnLayer(x,w,bias)
print(z)

#output: 284.9 

284.9


## Raw Kernels

- Raw kernels enables  the  direct use of kernels from CUDA source, and it is defined through the RawKernel class.
- The RawKernel object allows you to call the kernel with CUDA’s cuLaunchKernel interface. In other words, you have control over:
    - grid size
    - block size
    - shared memory size 
    - and stream. 
<img src="../images/cupy_kernel_memory.png">

In [29]:
import cupy as cp
add_kernel = cp.RawKernel(r'''
extern "C" __global__
void add_func(const float* x1, const float* x2, float* y) {
int tid = blockDim.x * blockIdx.x + threadIdx.x;
y[tid] = x1[tid] + x2[tid];
}
''', 'add_func')

N = 100
shape = (10, 10)

x1 = cp.arange(N, dtype=cp.float32).reshape(shape)
x2 = cp.arange(N, dtype=cp.float32).reshape(shape)
y = cp.zeros((shape), dtype=cp.float32)

add_kernel((10,), (10,), (x1, x2, y)) 

print(y)

#output:
#[[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]
#[ 20.  22.  24.  26.  28.  30.  32.  34.  36.  38.]
#[ 40.  42.  44.  46.  48.  50.  52.  54.  56.  58.]
#[ 60.  62.  64.  66.  68.  70.  72.  74.  76.  78.]
#[ 80.  82.  84.  86.  88.  90.  92.  94.  96.  98.]
#[100. 102. 104. 106. 108. 110. 112. 114. 116. 118.]
#[120. 122. 124. 126. 128. 130. 132. 134. 136. 138.]
#[140. 142. 144. 146. 148. 150. 152. 154. 156. 158.]
#[160. 162. 164. 166. 168. 170. 172. 174. 176. 178.]
#[180. 182. 184. 186. 188. 190. 192. 194. 196. 198.]]


[[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]
 [ 20.  22.  24.  26.  28.  30.  32.  34.  36.  38.]
 [ 40.  42.  44.  46.  48.  50.  52.  54.  56.  58.]
 [ 60.  62.  64.  66.  68.  70.  72.  74.  76.  78.]
 [ 80.  82.  84.  86.  88.  90.  92.  94.  96.  98.]
 [100. 102. 104. 106. 108. 110. 112. 114. 116. 118.]
 [120. 122. 124. 126. 128. 130. 132. 134. 136. 138.]
 [140. 142. 144. 146. 148. 150. 152. 154. 156. 158.]
 [160. 162. 164. 166. 168. 170. 172. 174. 176. 178.]
 [180. 182. 184. 186. 188. 190. 192. 194. 196. 198.]]


<img src="../images/raw_kernel.png">

#### Raw Kernels : Complex-value arrays

In [30]:
import cupy as cp

complex_kernel = cp.RawKernel(r'''
#include <cupy/complex.cuh>
extern "C" __global__
void my_func(const complex<float>* x1, const complex<float>* x2, complex<float>* y, float a){
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    y[tid] = x1[tid] + a * x2[tid];
}
''', 'my_func')

x1 = cp.arange(25, dtype=cp.complex64).reshape(5,5)
x2 = 1j*cp.arange(25, dtype=cp.complex64).reshape(5,5)
y = cp.zeros((5,5), dtype=cp.complex64)

complex_kernel((1,),(25,),(x1, x2,y,cp.float32(2.0)))
print(y)

#output:
#[[ 0. +0.j  1. +2.j  2. +4.j  3. +6.j  4. +8.j]
#[ 5.+10.j  6.+12.j  7.+14.j  8.+16.j  9.+18.j]
#[10.+20.j 11.+22.j 12.+24.j 13.+26.j 14.+28.j]
#[15.+30.j 16.+32.j 17.+34.j 18.+36.j 19.+38.j]
#[20.+40.j 21.+42.j 22.+44.j 23.+46.j 24.+48.j]]


[[ 0. +0.j  1. +2.j  2. +4.j  3. +6.j  4. +8.j]
 [ 5.+10.j  6.+12.j  7.+14.j  8.+16.j  9.+18.j]
 [10.+20.j 11.+22.j 12.+24.j 13.+26.j 14.+28.j]
 [15.+30.j 16.+32.j 17.+34.j 18.+36.j 19.+38.j]
 [20.+40.j 21.+42.j 22.+44.j 23.+46.j 24.+48.j]]


In [31]:
#This also produced the same output:
complex_kernel((5,), (5,), (x1, x2, y, cp.float32(2.0)))
print(y)

[[ 0. +0.j  1. +2.j  2. +4.j  3. +6.j  4. +8.j]
 [ 5.+10.j  6.+12.j  7.+14.j  8.+16.j  9.+18.j]
 [10.+20.j 11.+22.j 12.+24.j 13.+26.j 14.+28.j]
 [15.+30.j 16.+32.j 17.+34.j 18.+36.j 19.+38.j]
 [20.+40.j 21.+42.j 22.+44.j 23.+46.j 24.+48.j]]


In [32]:
####### Kernel Attributes ##################
print("max_dynamic_shared_size_bytes: ", complex_kernel.max_dynamic_shared_size_bytes )

print("max_threads_per_block: ", complex_kernel.max_threads_per_block )

print("attributes: ",complex_kernel.attributes)


max_dynamic_shared_size_bytes:  49152
max_threads_per_block:  1024
attributes:  {'max_threads_per_block': 1024, 'shared_size_bytes': 0, 'const_size_bytes': 0, 'local_size_bytes': 0, 'num_regs': 12, 'ptx_version': 75, 'binary_version': 75, 'cache_mode_ca': 0, 'max_dynamic_shared_size_bytes': 49152, 'preferred_shared_memory_carveout': -1}


### Raw Modules 

- The **RawModule** class is used to defining a large raw CUDA C source or loading an existing CUDA binary.
- It is initialized by a CUDA C source code having a several kernels (functions) such that needed kernels are retrieved by calling the **get_function()** method.

```python
import cupy as cp
loaded_from_source = r'''
extern "C" {
__global__ void test_sum(const float* A, const float* B, float* C, int N)
 { 
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if(tid < N)
    {
      C[tid] = A[tid] + B[tid]; 
    }
 }
 __global__ void test_multiply(const float* A, const float* B, float* C, int N )
 {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if(tid < N)
    {
        C[tid] = A[tid] * B[tid];
    }
 }
}'''
```
##### Example:

In [1]:
import cupy as cp

load_raw_module = r'''
extern "C" {
__global__ void sum_ker(const float* a, const float* b, float* c)
 { 
    unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
    c[tid] = a[tid] + b[tid]; 
    
 }
 __global__ void multiply_ker(const float* a, const float* b, float* c )
 {
    unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
    c[tid] = a[tid] * b[tid];
 }
}'''

module = cp.RawModule(code = load_raw_module)

ker_sum = module.get_function('sum_ker')
ker_times = module.get_function('multiply_ker')

a = cp.arange(25, dtype=cp.float32).reshape(5,5)
b = cp.ones((5,5), dtype=cp.float32)
c = cp.zeros((5,5), dtype=cp.float32)

In [3]:
# run the above cell before runing this cell
ker_sum((1,),(25,), (a,b,c))
print(c)

#output:
#[[ 1.  2.  3.  4.  5.]
#[ 6.  7.  8.  9. 10.]
#[11. 12. 13. 14. 15.]
#[16. 17. 18. 19. 20.]
#[21. 22. 23. 24. 25.]]

[[ 1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25.]]


In [5]:
ker_times((5,),(5,),(a,b,c))
print(c)


[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]


### Kernel Fusion

- Kernel fusion is a decorator that fuses functions. It can be used to define an elementwise or reduction kernels easily.


In [36]:
import cupy as cp

@cp.fuse(kernel_name='dnnlayerNode')
def dnnlayerNode(x, w, bias):
    return  (x * w) + bias
    
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
bias = cp.array([-0.5], dtype=cp.float32)
         
z = dnnlayerNode(x,w,bias)
print(z)

#output:
#[[-0.5  0.5  3.5]
#[ 8.5 15.5 24.5]
#[35.5 48.5 63.5]]

[[-0.5  0.5  3.5]
 [ 8.5 15.5 24.5]
 [35.5 48.5 63.5]]


## 6. Summary
<img src="../images/cupy_summary.png" width="80%" height="80%">

---
<div style="text-align:center; color:#FF0000"><b>Click on HPC Approach Link to view task on HPC serial code</b> </div>

## 7. [HPC Approach](serial_RDF.ipynb)

---

##  References
- https://docs.cupy.dev/en/stable/
- https://cupy.dev/
- CuPy Documentation Release 8.5.0, Preferred Networks, inc. and Preferred Infrastructure inc., Feb 26, 2021.
- Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018.
- Crissman Loomis and Emilio Castillo, CuPy Overview: NumPy Syntax Computation with Advanced CUDA Features, GTC Digital March, March 2020.
- https://www.gpuhackathons.org/technical-resources
- https://rapids.ai/start.html

--- 

## Licensing 

This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).